In [198]:
import numpy as np 
import pandas as pd 
#import os
from pyomo.environ import *
print('Libraries were improt was successfully!')


Libraries were improt was successfully!


In [199]:
def read_data(DataFile):
    xlsLoad = pd.ExcelFile(DataFile)
    Bus = pd.read_excel(xlsLoad, 'bus').set_index('id')
    Gen = pd.read_excel(xlsLoad, 'gen').set_index('id')
    Line = pd.read_excel(xlsLoad, 'line').set_index('id')
    Hour= pd.read_excel(xlsLoad, 'period_pead_data').set_index('Hour')
    #id should be a column with unique information. 
    
    return Bus, Gen, Line, Hour

In [200]:
def RTS(Bus, Gen, Line, Hour):
    # define a pyomo model
    model = ConcreteModel()
    N=Bus.index
    G=Gen.index
    K=Line.index
    H=Hour.index
   
    
    # define model variables and their limits
    model.p = Var(G, H, bounds=(0, None))
    model.f = Var(K,H)
    model.theta=Var(N,H)
     
    # Define the objective function
    model.obj = Objective(expr=sum(sum( model.p[g,h]*Gen.loc[g, 'vCost1'] for g in G)  for h in H))
    
    # define model constraints
    model.gen_max = ConstraintList()
    model.flow = ConstraintList()
    model.flow_max = ConstraintList()
    model.node_balance = ConstraintList()
    model.slack=ConstraintList()
    
    # Add the constraints to the model
      
    # Generation maximum Limits:
    for g in G: 
        for h in H:
            model.gen_max.add(model.p[g,h]<=Gen.loc[g,'CapHigh'])
    
    # Power flow equations and limits
    for k in K: 
        for h in H:
            model.flow.add(model.f[k,h]==Line.loc[k,'B_dc (MW)']*
                       (model.theta[Line.loc[k,'from'],h]-model.theta[Line.loc[k,'to'],h]))
            model.flow_max.add(model.f[k,h]<=Line.loc[k,'CAPA'])
            model.flow_max.add(model.f[k,h]>=-Line.loc[k,'CAPA'])
    
    # Slack bus constraint
    for h in H:
        model.slack.add(model.theta[1,h]==0)    
    # Node balance constraints
    for h in H:
        for n in N:
            demand = Bus.loc[n,'P-load']*Hour.loc[h,'Percent']/100
            inflow = 0
            outflow = 0
            localgen = 0
            for k in K:
                if Line.loc[k,'to']==n:
                    inflow += model.f[k,h]
                if Line.loc[k,'from']==n:
                    outflow += model.f[k,h]
            for g in G:
                if Gen.loc[g,'node']==n:
                    localgen += model.p[g,h]
            model.node_balance.add(localgen+inflow-outflow==demand)
    return model
    
    
   
    return model
    

In [201]:
# Read Data
#print(os.getcwd())
Bus, Gen, Line, Hour = read_data('RTS_Data.xlsx')
print("Data was read successfully.")
N=Bus.index
G=Gen.index
K=Line.index
H=Hour.index

# Construct Model
model = RTS(Bus, Gen, Line, Hour)
print("Construction of model complete.")

# Solving steps

# Identify the solver
opt = SolverFactory('glpk')

# Tell the model that you need the dual values too!
model.dual = Suffix(direction=Suffix.IMPORT)

# Solve the problem
Solution=opt.solve(model)

# Report the summary of solve
Solution.write(num=1)

# Report the output
for g in G:
    for h in H:
        print('Generator',g, 'in Hour' ,h, 'Output (MW):', model.p[g,h].value)
    
for k in K:
    for h in H:
        print('Line ',k,'in Hour',h,'Flow (MW):', model.f[k,h].value)

for n in N:
    for h in H:
        print('LMP for node ', n, 'in Hour',h, 'in ($/MWh) is: ', model.dual[model.node_balance[n,h]])
    
for k in K:
    for h in H:
        print('Flowgate price for line', k, 'in Hour' , h, 'in ($/MWh) in both directions are: '
          , model.dual[model.flow_max[2*k-1]],' and ', model.dual[model.flow_max[2*k]])


Data was read successfully.
Construction of model complete.
# ==========================================================
# = Solver Results                                         =
# ==========================================================
# ----------------------------------------------------------
#   Problem Information
# ----------------------------------------------------------
Problem: 
- Name: unknown
  Lower bound: 867594.678755507
  Upper bound: 867594.678755507
  Number of objectives: 1
  Number of constraints: 4129
  Number of variables: 2281
  Number of nonzeros: 7993
  Sense: minimize
# ----------------------------------------------------------
#   Solver Information
# ----------------------------------------------------------
Solver: 
- Status: ok
  Termination condition: optimal
  Statistics: 
    Branch and bound: 
      Number of bounded subproblems: 0
      Number of created subproblems: 0
  Error rc: 0
  Time: 0.2541186809539795
# ---------------------------------

KeyError: "Index '(1, 1)' is not valid for indexed component 'node_balance'"